In [1]:
from architectures_v1 import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import os
import numpy as np
import tensorflow as tf
import pandas as pd
import time

## Loading Training/Testing/Validation Data

In [2]:
dataset = "ChlorineConcentration"

x_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_train.npz')))
x_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_test.npz')))

x_train = np.reshape(x_train_load['arr_0'], [x_train_load['arr_0'].shape[0], x_train_load['arr_0'].shape[1], 1])
x_test = np.reshape(x_test_load['arr_0'], [x_test_load['arr_0'].shape[0], x_test_load['arr_0'].shape[1], 1])

x_all = np.concatenate((x_train, x_test), axis = 0)

n_instances = x_all.shape[0]

y_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_train.npz')))
y_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_test.npz')))

y_train = y_train_load['arr_0']
y_test = y_test_load['arr_0']

y_all = np.concatenate((y_train, y_test), axis = 0)

x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size = 0.3, random_state = 42)

x_valid = x_test[:int(0.1*n_instances), :, :]
y_valid = y_test[:int(0.1*n_instances)]

x_test = x_test[int(0.1*n_instances):, :, :]
y_test = y_test[int(0.1*n_instances):]

print(f"x_train: {x_train.shape} - y_train: {y_train.shape}")
print(f"x_test: {x_test.shape} - y_test: {y_test.shape}")
print(f"x_valid: {x_valid.shape} - y_valid: {y_valid.shape}")

x_train: (3014, 166, 1) - y_train: (3014,)
x_test: (863, 166, 1) - y_test: (863,)
x_valid: (430, 166, 1) - y_valid: (430,)


In [6]:
seed = 42
tf.random.set_random_seed(seed)
np.random.seed(seed)

length = x_train.shape[1]
n_features = x_train.shape[-1]

model = Sequential()

model.add(RNN(OrthogonalCell(units = 212,
                             order = 50,
                             variant = 'ct_laguerre',
                             dt = 1,
                            input_dims = 1), 
              input_shape = (length, n_features),
             return_sequences = False))
model.add(Dense(to_categorical(y_train).shape[-1], activation = "softmax"))

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn_1 (RNN)                  (None, 212)               58571     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 852       
Total params: 59,423
Trainable params: 56,871
Non-trainable params: 2,552
_________________________________________________________________


In [7]:
if not os.path.exists(os.path.abspath(os.path.join('models', dataset))):
    os.mkdir(os.path.abspath(os.path.join('models', dataset)))

file_path = os.path.abspath(os.path.join('models', dataset, f'{dataset}-laguerre.hdf5'))

callbacks = [
    ModelCheckpoint(filepath=file_path, monitor='val_loss', save_best_only = True, mode = 'min', verbose = 1), 
    EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min')]

t = time.time()

result = model.fit(x_train, 
                   to_categorical(y_train),
                   epochs = 200, 
                   batch_size = 100, 
                   validation_data = (x_valid, to_categorical(y_valid)), 
                   callbacks = callbacks)

print(f"Training time: {time.time() - t} s")

df_results = pd.DataFrame(result.history)
df_results.to_csv(os.path.abspath(os.path.join('models', dataset, 'laguerre_results.csv')))

Train on 3014 samples, validate on 430 samples
Epoch 1/200
3000/3014 [============================>.] - ETA: 0s - loss: 1.0441 - acc: 0.5290
Epoch 00001: val_loss improved from inf to 1.00791, saving model to /Users/sergio.dorado/Documents/GitHub_Repositories/01_Maintained/NeuralODE/models/ChlorineConcentration/ChlorineConcentration-laguerre.hdf5
3014/3014 [==============================] - 4s 1ms/sample - loss: 1.0448 - acc: 0.5279 - val_loss: 1.0079 - val_acc: 0.5977
Epoch 2/200
3000/3014 [============================>.] - ETA: 0s - loss: 0.9937 - acc: 0.5453
Epoch 00002: val_loss improved from 1.00791 to 0.92227, saving model to /Users/sergio.dorado/Documents/GitHub_Repositories/01_Maintained/NeuralODE/models/ChlorineConcentration/ChlorineConcentration-laguerre.hdf5
3014/3014 [==============================] - 4s 1ms/sample - loss: 0.9934 - acc: 0.5455 - val_loss: 0.9223 - val_acc: 0.5977
Epoch 3/200
3000/3014 [============================>.] - ETA: 0s - loss: 0.9734 - acc: 0.5567
E

In [8]:
saved_epoch = np.argmin(result.history['val_loss'])

print(f"Epoch {saved_epoch}: {result.history['val_acc'][saved_epoch]} val_acc")

model.load_weights(file_path)
model.evaluate(x_test, to_categorical(y_test))

Epoch 94: 0.9883720874786377 val_acc
863/863 [==============================] - 1s 621us/sample - loss: 0.0321 - acc: 0.9907


[0.03214570412440344, 0.99073]